## Styl Kodu

Ogólnie przyjęte zasaday co do jakości kodu są opisane w PEP8. Funkcjonują moduły Pythonowe, które sprawdzają styl kodu i wskazują gdzie jakie błedy zostały popełnione. Te moduły to:

- flake8
- pep8
- pylint


do sortowania importów słyży *isort*

## Tuple unpacking

In [ ]:
a, b = 1, 2  # a=1, b=2
a, b = (1, 2)  # a=1, b=2

# Python 3
a, b, *_ = (1, 2, 3, 4, 5)
print(a, b, _)

a, *_, b = (1, 2, 3, 4, 5)
print(a, b, _)

*_, a,  b = (1, 2, 3, 4, 5)
print(a, b, _)

In [ ]:
# Python 3.5+
# można używać tuple i dict unpacking do tworzenia innych list i słowników

d1 = {1:11, 2:22}
d2 = {3:33, 4:44}

d3 = {**d1, **d2}
print(d3)

print('>>><<<')

l1 = [1, 2, 3]
l2 = [4, 5, 6]

l3 = [*l1, *l2]
print(l3)

print('>>><<<')
l4 = [*d1]  # rozpakowuje tylko klucze do listy
print(l4)

## Trikowe rzeczy

### "kopiowanie" krotki

In [ ]:
x = (1, 2, 3, 4)
y = tuple(x)  # kopiowanie
x is y  # nie skopiowało

### tworzenie listy list na skróty

In [ ]:
x = [[]] * 5  # lista 5ciu list
print(x)

x[0].append(3)
print(x)  # dodało się wszędzie - Python zamiast stworzyć 5 list, stworzył 1
         # i skopiował referencje do niej

print('---')
# poprawny sposób
x = [[] for i in range(5)]
print(x)

x[0].append(3)
print(x)  # teraz działa

### hash ma znaczenie

In [ ]:
print(hash(True), hash(1), hash(1.0))

print(set([True, 1, 1.0]))

print({True: 1, 1: 2, 1.0: 3})

print('---')
print(True in {1})

### funkcja iter i usuwanie elementów podczas iteracji

In [ ]:
print('--- używanie iteratora do obiektu - iter()')
x = [1, 2, 3, 4, 5]
ix = iter(x)
print(ix)
print(next(ix))
print(next(ix))
print(next(ix))
print(next(ix))
print(next(ix))

print()
print('--- iterowanie aż do napotkania wartości kończącej')
# inne zastosowanie iter - iter(funkcja, obiekt) - funkcja wykonuje sie tak długo,
# aż napotka <obiekt>
from functools import partial
from random import randint

x = iter(partial(randint, 1, 5), 4)
print(list(x))  # przy każdym uruchomieniu będzie inny wynik bo losuje liczby aż napotka 4

In [ ]:
# USUWANIE ELEMENTÓW PODCZAS ITERACJI
print('--- usuwanie elementów 2, 3, 6 z listy podczas iteracji')
# usuwanie elementów podczas iteracji
x = list(range(10))
for i, item in enumerate(x):
    print(item)
    if item in (2, 3, 6):  # chcemy usunąć liczby 2, 3, 6
        print('popped: ', x.pop(i))  # 3 NIE zostaje usunięte
print("WNIOSEK: 3 zostaje pominięte")

# w printach zostaną pominięte 2 i 5 - lista zostaje "przesunięta" dwa razy -
# najpierw 2 wskakuje na miejsce 1 a potem 5 na miejsce 4

In [ ]:
# OBEJŚCIE?
print('--- usuwanie elementów 2,3,6 z listy podczas iteracji - płytka kopia')
x = list(range(10))
for i, item in enumerate(list(x)):  # płytka kopia - nie iterujemy po modyfikowanej liście
    print(item)
    if item in (2, 3, 6):
        print('popped: ', x.pop(i))  # usuwa 2, 4 i 8 - JESZCZE GORZEJ!!
                                     # wcześniej przynajmniej nie usuwało tego co nie trzeba
print("WNIOSEK: zostają usuniete nie te elementy co trzeba")

In [ ]:
# WNIOSEK: Nie modyfikować listy po której się iteruje
# alternatywa: list comprehension
print('--- usuwanie elementów 2,3,6 z listy podczas iteracji - list comprehension')
x = list(range(10))
x2 = [i for i in x if i not in (2, 3, 6)]
print(x2)
print("WNIOSEK: efekt jest poprawny, ale mamy nową listę w pamięci")
print('W zależności od potrzeb, można zastąpić listę generatorem i oszczędzić trochę miejsca i czasu')

### Asercje

nie nalezy dodawać nawiasów po assert

In [ ]:
assert (False, "Error message")  # przejdzie - bo interpretuje jako krotkę dwuelementową

assert False, "Error Message 2"

### AST

Python udostępnia moduł ast, który pozwala rozbić kod pythonowy na drzewo składniowe

In [ ]:
from __future__ import print_function
import ast

code = """
x = 23

class A(object):
    def __init__(self):
        self.attr = 33
        
        
for i in range(4):
    print i

def f(x):
    return x ** 2
"""

tree = ast.parse(code)  # aprsujemy kod źródłowy jako drzewo
print('content of tree: ', dir(tree))
print()

print('tree.body: ', tree.body)  # body tego elementu (oznaczającego "plik" code) zawiera przypisanie, definicję klasy,
                  # pętle for i definicję fuknkcji
print()
    
assign, classdef, forloop, functiondef = tree.body

print('assign ', dir(assign))
print()
print('classdef ', dir(classdef))
print()
print('forloop ', dir(forloop))
print()
print('functiondef ', dir(functiondef))

Generalnie modułu ast można używać do modyfikowania "kodu źródłowego" (bo to juz jest zinterpretowany kod a nie kod w formie pliku) w locie

### wrzucanie pliku .pth do site-packages

do katalogu site-packages w lib/python.../site-packages można wrzucić plik z rozszerzeniem .pth to każda linijka zaczynająca się od "import" zostanie wykonana (funkcją exec) przy uruchomieniu interpretera

### modyfikowanie sys.path
lepiej tego unikać, może to prowadzić do ciężkich do wykrycia błędów - przykład w examples/dodatkowe/manipulacja_path